<a href="https://colab.research.google.com/github/pstorniolo/Master2021/blob/main/2021_11_02_Spark_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MLlib

**MLlib** è la libreria di machine learning (ML) di Spark. Il suo obiettivo è rendere l'apprendimento automatico pratico scalabile e facile. Fornisce strumenti ad alto livello come:

*   Algoritmi ML: algoritmi di apprendimento comuni come *classification, regression, clustering, and collaborative filtering*;
*   Pipeline: strumenti per la costruzione, la valutazione e l'ottimizzazione delle pipeline ML;
*   Caratterizzazione: *feature extraction, transformation, dimensionality reduction, and selection*;
*   Persistenza: salvataggio e caricamento di algoritmi, modelli e pipeline
*   Utility: algebra lineare, statistica, gestione dati, ecc.

https://spark.apache.org/docs/latest/api/python/reference/pyspark.ml.html


In [ ]:
!curl ipecho.net/plain

In [ ]:
# Install Spark 3.2.0 - JDK11
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!curl -O https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
#!curl -O http://www.pa.icar.cnr.it/storniolo/files/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!rm -f *.tgz

import os
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

!pip -q install findspark
#!pip install -q dnspython

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
#from pymongo import MongoClient

In [ ]:
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

print(spark.version)
#spark.stop()

Creo un collegamento con spark per semplificare il percorso dei dati

In [ ]:
!ln -s spark-3.2.0-bin-hadoop3.2 spark
!ls -la

#Classification

##Logistic regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Load training data
training = spark.read.format("libsvm").load("spark/data/mllib/sample_libsvm_data.txt")
training.printSchema()
training.show(truncate=False)

In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [ ]:
# Fit the model
lrModel = lr.fit(training)

In [ ]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

##Multinomial logistic regression

La *Multiclass classification* è supportata tramite regressione logistica multinomiale (softmax). Nella regressione logistica multinomiale, l'algoritmo produce ***K*** insiemi di coefficienti o una matrice di dimensione ***K×J*** dove ***K*** è il numero di classi di risultati e ***J*** è il numero di caratteristiche.

In [ ]:
# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

In [ ]:
# Fit the model
mlrModel = mlr.fit(training)

In [ ]:
# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

##Multilayer perceptron classifier

Il MultiLayer Perceptron Classifier (MLPC) è un classificatore basato sulla rete neurale artificiale *feedforward*. MLPC è costituito da più livelli di nodi. Ogni livello è completamente connesso al livello successivo nella rete. I nodi nel livello di input rappresentano i dati di input. Tutti gli altri nodi mappano gli input agli output mediante una combinazione lineare degli input con i pesi ***w*** e bias ***b*** del nodo e applicando una **funzione di attivazione**.

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = spark.read.format("libsvm").load("spark/data/mllib/sample_multiclass_classification_data.txt")

data.printSchema()
data.show(truncate=False)

In [ ]:
# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]
print(data.count(),train.count(),test.count())

In [ ]:
# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [4, 6, 5, 3]

In [ ]:
# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=150, layers=layers, blockSize=20, seed=1234)

In [ ]:
# train the model
model = trainer.fit(train)

In [ ]:
# compute accuracy on the test set
result = model.transform(test)
result.printSchema()

In [ ]:
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

In [ ]:
evaluator = MulticlassClassificationEvaluator(metricName="hammingLoss")
print("Test set hammingLoss = " + str(evaluator.evaluate(predictionAndLabels)))

##Linear Support Vector Machine

Una *support vector machine* costruisce un iperpiano o un insieme di iperpiani in uno spazio ad alta o infinita dimensione, che può essere utilizzato per la classificazione, la regressione o altre attività. Intuitivamente, una buona separazione è ottenuta dall'iperpiano che ha la distanza maggiore dai punti dati di addestramento più vicini di qualsiasi classe (cosiddetto margine funzionale), poiché in generale maggiore è il margine, minore è l'errore di generalizzazione del classificatore. **LinearSVC** in Spark ML supporta la classificazione binaria con SVM lineare.

In [ ]:
from pyspark.ml.classification import LinearSVC

# Load training data
training = spark.read.format("libsvm").load("spark/data/mllib/sample_libsvm_data.txt")

In [ ]:
lsvc = LinearSVC(maxIter=100, regParam=0.1)

In [ ]:
# Fit the model
lsvcModel = lsvc.fit(training)

In [ ]:
# Print the coefficients and intercept for linear SVC
print("Coefficients: " + str(lsvcModel.coefficients))
print("Intercept: " + str(lsvcModel.intercept))

In [ ]:
#!ls -la spark/examples/src/main/python/ml/

#Clustering

##K-means

**k-means** è uno degli algoritmi di clustering più comunemente usati che raggruppa i punti dati in un numero predefinito di cluster. L'implementazione di MLlib include una variante parallelizzata del metodo ***k-means++*** chiamato ***kmeans||***.

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
dataset = spark.read.format("libsvm").load("spark/data/mllib/sample_kmeans_data.txt")
dataset.printSchema()
dataset.show(truncate=False)

In [ ]:
# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

In [ ]:
# Make predictions
predictions = model.transform(dataset)
predictions.printSchema()
predictions.show(truncate=False)

In [ ]:
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

In [ ]:
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

In [ ]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

##RDD-based K-means

In [ ]:
from numpy import array
from math import sqrt

from pyspark.mllib.clustering import KMeans, KMeansModel

# Load and parse the data
data = sc.textFile("spark/data/mllib/kmeans_data.txt")

In [ ]:
!cat spark/data/mllib/kmeans_data.txt

In [ ]:
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

In [ ]:
# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

In [ ]:
# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [ ]:
WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

In [ ]:
!rm -rf KMeansModel

In [ ]:
# Save and load model
clusters.save(sc, "KMeansModel")
sameModel = KMeansModel.load(sc, "KMeansModel")

In [ ]:
!ls -la KMeansModel

#Frequent Pattern Mining



##FP-Growth

L'algoritmo **FP-growth** è descritto nel documento "*Han et al., Mining frequent patterns without candidate generation*", dove "FP" sta per Frequent Pattern. Dato un set di dati di transazioni, il primo passo della crescita del FP-growth è calcolare le frequenze degli articoli e identificare gli articoli frequenti.

Diversamente dagli algoritmi *Apriori-like* progettati per lo stesso scopo, il secondo passaggio di FP-growth utilizza una struttura ad albero dei suffissi (FP-tree) per codificare le transazioni senza generare esplicitamente insiemi candidati, che di solito sono onerosi da generare. Dopo il secondo passaggio, gli insiemi di elementi frequenti possono essere estratti dall'albero FP.

In spark.mllib, si è implementata una versione parallela di FP-growth chiamata PFP, come descritto in "*Li et al., PFP: Parallel FP-growth for query recommendation*". PFP distribuisce il lavoro di crescita di alberi FP in base ai suffissi delle transazioni e quindi è più scalabile di un'implementazione su una singola macchina.

In [ ]:
from pyspark.ml.fpm import FPGrowth

df = spark.createDataFrame([
    (0, [1, 2, 5]),
    (1, [1, 2, 3, 5]),
    (2, [1, 2])
], ["id", "items"])

df.printSchema()
df.show()

In [ ]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(df)

In [ ]:
# Display frequent itemsets.
model.freqItemsets.show()

In [ ]:
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(df).show()

In [ ]:
# Display generated association rules.
model.associationRules.show()

##RDD-based FP-growth

In [ ]:
from pyspark.mllib.fpm import FPGrowth

data = sc.textFile("spark/data/mllib/sample_fpgrowth.txt")
print(data.collect())

In [ ]:
transactions = data.map(lambda line: line.strip().split(' '))

model = FPGrowth.train(transactions, minSupport=0.2, numPartitions=10)

result = model.freqItemsets().collect()

In [ ]:
for fi in result:
    print(fi)

#ML Tuning (model selection)

##Cross-Validation

**CrossValidator** inizia suddividendo il set di dati in un insieme di *fold* che vengono utilizzate come set di dati di addestramento e test separati. Ad esempio, con ***k=3*** fold, CrossValidator genererà 3 coppie di set di dati (addestramento, test), ognuna delle quali utilizza 2/3 dei dati per l'addestramento e 1/3 per il test. Per valutare un particolare *ParamMap*, *CrossValidator* calcola la metrica di valutazione media per i 3 modelli prodotti adattando l'estimatore alle 3 diverse coppie di set di dati (addestramento, test).
Dopo aver identificato la migliore *ParamMap*, *CrossValidator* infine riadatta l'estimatore utilizzando la migliore *ParamMap* e l'intero set di dati.

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Prepare training documents, which are labeled.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 2.0),
    (4, "b spark who", 1.0),
    (5, "g d a y", 0.0),
    (6, "spark fly", 1.0),
    (7, "was mapreduce", 0.0),
    (8, "e spark program", 1.0),
    (9, "a e c l", 0.0),
    (10, "spark compile", 1.0),
    (11, "hadoop software", 2.0)
], ["id", "text", "label"])

training.show()

In [ ]:
# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10)

pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

Ora usiamo *Pipeline* come un *Estimator*, facendone il *wrapping* in un'istanza *CrossValidator*. Questo ci consentirà di scegliere congiuntamente i parametri per tutte le fasi della pipeline. Un CrossValidator richiede un Estimator, un set di Estimator ParamMaps e un Evaluator. Usiamo il metodo *ParamGridBuilder* per costruire una griglia di parametri su cui cercare. Con 3 valori per *hashingTF.numFeatures* e 2 valori per *lr.regParam*, questa griglia avrà 3 x 2 = 6 impostazioni dei parametri tra cui *CrossValidator* potrà scegliere.

In [ ]:
paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

for i in paramGrid:
  print(i)

In [ ]:
#evaluator = BinaryClassificationEvaluator()
evaluator = MulticlassClassificationEvaluator()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)

In [ ]:
# Prepare test documents, which are unlabeled.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "mapreduce spark"),
    (7, "ape hadoop")
], ["id", "text"])

test.show()

In [ ]:
# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(test)
prediction.printSchema()

prediction.select("id", "text", "probability", "prediction").show(truncate=False)

training.show()

##Train-Validation Split

Oltre a *CrossValidator* Spark offre anche *TrainValidationSplit* per l'ottimizzazione degli iperparametri. *TrainValidationSplit* valuta ogni combinazione di parametri solo una volta, invece di k volte come nel caso di CrossValidator. È, quindi, meno oneroso, ma non produrrà risultati altrettanto affidabili quando il set di dati di addestramento non è sufficientemente grande.

A differenza di *CrossValidator*, *TrainValidationSplit* crea una singola coppia di set di dati (addestramento, test). Divide il set di dati in queste due parti utilizzando il parametro *trainRatio*. Ad esempio con *trainRatio=0.75*, *TrainValidationSplit* genererà una coppia di set di dati di addestramento e test in cui il 75% dei dati viene utilizzato per l'addestramento e il 25% per la convalida.

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

# Prepare training and test data.
data = spark.read.format("libsvm").load("spark/data/mllib/sample_linear_regression_data.txt")
data.printSchema()
print(data.count())
data.show(5,truncate=False)

train, test = data.randomSplit([0.9, 0.1], seed=12345)
print(train.count())
train.show(2)
print(test.count())
test.show(2)

In [ ]:
lr = LinearRegression(maxIter=10)

In [ ]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

In [ ]:
# In this case the estimator is simply the linear regression.
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           trainRatio=0.8) # 80% of the data will be used for training, 20% for validation.

In [ ]:
# Run TrainValidationSplit, and choose the best set of parameters.
model = tvs.fit(train)

In [ ]:
# Make predictions on test data. model is the model with combination of parameters
# that performed best.
result = model.transform(test)
result.printSchema()
result.select("features", "label", "prediction").show(truncate=False)


#Stop Session

In [ ]:
#spark.stop()